# Annotate ecosystem observations
Two ways:
- Using existing repositories inat existing information is unstructure, requires additional information from experts/user
- structured approach to capture new records. Flexibility: quick input of minimal observation, but allow more detailed input from users


First filter geographic (100km radius...) and thematic (realm/biome) 

request list of potential EFG from database (call to lambda function?)

We are in Rwanda so let's get the list of potential ecosystems from https://global-ecosystems.org/analyse?regionId=ADM_122,

In [2]:
import pandas as pd
potential_ecos = pd.read_csv("~/Desktop/Rwanda-July-2022/data/rle-query-results_20220613_1655105198171.csv")

In [46]:
biomes_avail = potential_ecos[['Realm ID','Biome ID','Biome']].drop_duplicates()

In [47]:
def biome_selection(avail): 
    selected=list()
    print("These are the potential realms and biomes potentially present at your location")
    print(avail)

    while True:

        res = input("Select one Biome ID to add to the list, or a Realm ID to select multiple biomes .\n Select X to finish ")

        if res in avail['Biome ID'].values:
            selected.append(res)
            print("You added %s, your list is now %s" % (res,"; ".join(selected)))
        if res in avail['Realm ID'].values:
            biomes=avail.loc[avail['Realm ID']==res]['Biome ID'].values.tolist()
            selected.extend(biomes)
            print("You added all biomes matching Realm ID %s, your list is now %s" % (res,"; ".join(selected)))

        elif res == "X":
            if len(selected)>0:
                print("Your final selection is %s" % (";".join(selected)))
            print("Let's go to the next question")
            return(selected)
            break
        else:
            print("Did not understand the response, try again.")
        

In [48]:
biomes_selected=biome_selection(biomes_avail)


These are the potential realms and biomes potentially present at your location
   Realm ID Biome ID                                   Biome
0         F       F1                Rivers and streams biome
3         F       F2                             Lakes biome
5         F       F3               Artificial wetlands biome
8         S       S1               Subterranean lithic biome
10        S       S2  Anthropogenic subterranean voids biome
11        T       T1      Tropical-subtropical forests biome
12        T       T3  Shrublands and shrubby woodlands biome
13        T       T4           Savannas and grasslands biome
14        T       T6          Polar/alpine (cryogenic) biome
15        T       T7                Intensive land-use biome
20       SF      SF1          Subterranean freshwaters biome
21       TF      TF1               Palustrine wetlands biome


Select one Biome ID to add to the list, or a Realm ID to select multiple biomes .
 Select X to finish  T


You added all biomes matching Realm ID T, your list is now T1; T3; T4; T6; T7


Select one Biome ID to add to the list, or a Realm ID to select multiple biomes .
 Select X to finish  X


Your final selection is T1;T3;T4;T6;T7
Let's go to the next question


In [72]:
def efg_selection(potential,biomes):
    avail=potential[potential['Biome ID'].isin(biomes)][['ID','Group']]
    discard=list()
    print("These are the ecosystem functional groups potentially present at your location")
    print(avail)

    while True:
        res = input("Select one ID to remove from the list.\n Select X to finish ")
        if res in avail['ID'].values:
            discard.append(res)
            print("You want to remove %s, your list of ecosystems to remove is now %s" % (res,"; ".join(discard)))
        elif res == "X":
            if len(discard)>0:
                x=avail['ID'].values
                selected=x[x not in discard]
                print("Your final selection is %s" % (";".join(selected)))
            print("Let's go to the next question")
            return(selected)
            break
        else:
            print("Did not understand the response, try again.")
        

In [73]:
efg_selected=efg_selection(potential_ecos,biomes_selected)


These are the ecosystem functional groups potentially present at your location
      ID                                         Group
11  T1.3      Tropical/Subtropical montane rainforests
12  T3.4  Young rocky pavements, lava flows and screes
13  T4.1                              Trophic savannas
14  T6.5     Tropical alpine grasslands and herbfields
15  T7.1                              Annual croplands
16  T7.2                      Sown pastures and fields
17  T7.3                                   Plantations
18  T7.4               Urban and industrial ecosystems
19  T7.5  Derived semi-natural pastures and old fields


Select one ID to remove from the list.
 Select X to finish  T7.4


You want to remove T7.4, your list of ecosystems to remove is now T7.4


Select one ID to remove from the list.
 Select X to finish  X


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [71]:
efg_selected

['T7.4']

In [43]:
biomes_avail.loc[biomes_avail['Realm ID']=='T']['Biome ID'].values.tolist()

['T1', 'T3', 'T4', 'T6', 'T7']

In [66]:
any(potential_ecos['ID'].isin(['T1.3']))

True

## Checklist of features

We can create a function to allow user to select features from a list to narrow down candidates. 

We will use a nested structure to work on this list:

In [33]:
FeatureGroups={'Vegetation structure':list(),
               'Animal traits':list(),
               'Plant traits':list(),
               'Landscape features':list(),
               'Human activity':list()}

This are the questions and answers for Vegetation structure

In [46]:
FeatureGroups['Vegetation structure']=[{'question':'Dominant growth form (excluding scattered emergent plants)',
        'options':['Tree','Shrub','Herb','Grass','Moss or lichen']},
        {'question':'Cover of dominant vegetation layer (exclude emergents)',
        'options':['Dense (>50%)','Open(30-50%)','Sparse(10-30%)','Very sparse (<10%)']},
        {'question':'Tree canopy height',
        'options':['Very tall (<40m)','Tall (30-40m)','Medium(10-30m)','Low(2-10m)','Very low (<2m)']},
        {'question':'Cover of bare ground (no vegetation, no litter)',
        'options':['Most(>50%)','Common (25-50%)','Limited(<25%)']}]

This are the questions and answers for Human activity

In [47]:
FeatureGroups['Human activity']=[{'question':'Cultivated crops (<2 yr rotation)',
        'options':['Dominant','Conspicuous but not dominant','Not common']},
{'question':'Artificial pastures',
        'options':['Dominant','Conspicuous but not dominant','Not common']},
{'question':'Domestic livestock',
        'options':['Dominant','Conspicuous but not dominant','Not common']},
{'question':'Irrigation infrastructure',
        'options':['Intensive','Conspicuous but not intensive','Not observed']},
{'question':'Plantations of woody plants',
        'options':['Dominant','Conspicuous but not dominant','Not common']},
{'question':'Urban/industrial infrastructure',
        'options':['Dominant','Conspicuous but not dominant','Not common']},
{'question':'Parks/gardens in urban landscapes',
        'options':['Dominant','Conspicuous but not dominant','Not common']},
{'question':'Refuse dumps & excavations',
        'options':['Dominant','Conspicuous but not dominant','Not common']}]


This are the questions and answers for Animal traits

In [35]:
FeatureGroups['Animal traits']=[{'question':'Large herbivores',
        'options':['Dominant','Conspicuous but not dominant','Not common']},
{'question':'Large carnivores',
        'options':['Dominant','Conspicuous but not dominant','Not common']},
{'question':'Arboreal primates',
        'options':['Dominant','Conspicuous but not dominant','Not common']},
{'question':'Other arboreal mammals',
        'options':['Evidence of presence','not detected']},
{'question':'Seabirds or marine mammals',
        'options':['Dominant','Conspicuous but not dominant','Not common']},
{'question':'Wetland birds',
        'options':['Dominant','Conspicuous but not dominant','Not common']},
{'question':'Semi-aquatic mammals, reptiles & amphibians',
        'options':['Dominant','Conspicuous but not dominant','Not common']}]


This are the questions and answers for Landscape features

In [48]:
FeatureGroups['Landscape features']=[{'question':'Surface rock cover (includes rocks covered in moss/lichen)',
        'options':['Dominan (>50% cover)','intermediate (5-10% cover)','Not common']},
{'question':'Landform type',
        'options':['Depositional flat','lower slope (deposition))','upper slope (erosion)','summit or ridge','plateau']},
{'question':'Slope',
        'options':['Flat (<2 deg)','Gentle (2-10deg)','Steep(10-25)','Very steep(>25)']},
{'question':'Surface inundation',
        'options':['Surface water present and likely persistent >1 month','Otherwise']},
{'question':'Groundwater',
        'options':['Frequently boggy (groundwater close to surface)','Otherwise']},
{'question':'Snow cover',
        'options':['Present and >10cm deep','None or <10cm deep']},
{'question':'Permanent ice',
        'options':['Present','absent']},
{'question':'Fire',
        'options':['Evidence of fire in recent years','Evidence of historical fire (charcoal scars)','no evidence of fire']},
{'question':'Shoreline proximity',
        'options':['Within 20m','Within 200m','>200m']},
{'question':'Stream proximity',
        'options':['Within 10m','Within 100m','>100m']}]

each question has multiple answers, each answer related to multiple EFGs so question will be relevant when at least two of those are present

same biome and same area -> narrowing down questions to discriminate those
questions that discriminate between realms, questions that discriminate between biomes

Here we create a function to iterate through the questions and answers:

In [59]:
def select_response(Q,A,multiple=False):
    print(Q)
    for k in range(len(A)):
        print("(%s) %s" %(k,A[k]))
        if multiple:
            result=list()
            mssg='select one or more response(s), X to finish'
        else:
            result=None
            mssg='select your response, X to cancel'
    while True:
        slc=input(mssg)
        if slc.isnumeric():
            slc=int(slc)
        if slc in range(len(A)):
            if multiple:
                result.append(A[slc])
            else:
                result=A[slc]
                break
        elif slc == "X":
            print("Ok, go back")
            break
        else:
            print("Did not understand the response, try again.")
    if result is not None and len(result)>0:
            return(result)

In [61]:
select_response(FeatureGroups['Landscape features'][0]['question'],
                FeatureGroups['Landscape features'][0]['options'],multiple=False)

Surface rock cover (includes rocks covered in moss/lichen)
(0) Dominan (>50% cover)
(1) intermediate (5-10% cover)
(2) Not common


select your response, X to cancel 2


'Not common'

In [63]:
def collect_responses(QA):
    responses=dict()
    while True:
        for j in range(len(QA)):
            if QA[j]['question'] not in responses.keys():
                print("(%s) %s" %(j,QA[j]['question']))
        qst=input('select which feature to include, X to exit this menu')
        if qst.isnumeric():
            qst=int(qst)
        if qst in range(len(QA)):
            res=select_response(QA[qst]['question'],QA[qst]['options'])
            if res is not None:
                responses[QA[qst]['question']]=res
            #print(QA[qst]['question'])
            #opt=QA[qst]['options']
            #for k in range(len(opt)):
            #    print("(%s) %s" %(k,opt[k]))
            #slc=input('select your response, X to cancel')
            #if slc.isnumeric():
            #    slc=int(slc)
            #if slc in range(len(opt)):
            #    responses[QA[qst]['question']]=opt[slc]
            #elif qst == "X":
            #    print("Ok, go back")
            #    break
            #else:
            #    print("Did not understand the response, try again.")
        elif qst == "X":
            print("Let's go to the next question")
            break
        else:
            print("Did not understand the response, try again.")
    return(responses) 

In [64]:
def annotate(features):
    responses=dict()
    feat_grp = list(features.keys())
    while True:
        for j in range(len(feat_grp)):
            print("(%s) %s" %(j,feat_grp[j]))
        qst=input('select one feature group to review next, or X to finish')
        if qst.isnumeric():
            qst=int(qst)
        if qst in range(len(feat_grp)):
            ans=feat_grp[qst]
            print("You selected %s:" % ans)
            if len(features[ans])>0:
                responses[ans]=collect_responses(features[ans])
            else:
                print("No further questions for now")
        elif qst == "X":
            print("Ok, Thanks!")
            break
        else:
            print("Did not understand the response, try again.")
    return(responses)

In [65]:
annotate(FeatureGroups)

(0) Vegetation structure
(1) Animal traits
(2) Plant traits
(3) Landscape features
(4) Human activity


select one feature group to review next, or X to finish 4


You selected Human activity:
(0) Cultivated crops (<2 yr rotation)
(1) Artificial pastures
(2) Domestic livestock
(3) Irrigation infrastructure
(4) Plantations of woody plants
(5) Urban/industrial infrastructure
(6) Parks/gardens in urban landscapes
(7) Refuse dumps & excavations


select which feature to include, X to exit this menu 0


Cultivated crops (<2 yr rotation)
(0) Dominant
(1) Conspicuous but not dominant
(2) Not common


...

select which feature to include, X to exit this menu X


Let's go to the next question
(0) Vegetation structure
(1) Animal traits
(2) Plant traits
(3) Landscape features
(4) Human activity


select one feature group to review next, or X to finish X


Ok, Thanks!


{'Human activity': {'Cultivated crops (<2 yr rotation)': 'Dominant',
  'Domestic livestock': 'Dominant',
  'Refuse dumps & excavations': 'Conspicuous but not dominant'},
 'Animal traits': {'Other arboreal mammals': 'Evidence of presence'}}